In [1]:
%config Completer.use_jedi = False

In [119]:
import matplotlib.pyplot as plt
import glob
import pandas as pd
import numpy as np
import csv
from ast import literal_eval


 
def set_table_indexes(algorithm_logs):
    names = list(filter(lambda x: "Final" in x,algorithm_logs.keys()))
    indexes={0:"Total distance (km)", 1:"Total load (kg)", 2:"Total Estimated Fuel Consumption (L) (Hao et al.)",
      3:"Avg Estimated Fuel Conspumtion (L/100km) (Hao et al.)",4:"Total Estimated Fuel Consumption (L) (Rakha et al.)",
      5:"Avg Estimated Fuel Conspumtion (L/100km) (Rakha et al.)",
      6:"Avg Speed (km/h)",7:"Computation Time (s)"}

    for name in names:
        algorithm_logs[name].rename(index=indexes,inplace=True)
    return algorithm_logs



def load_logs(folder_path,filetype="csv"):
    logdir = "../logs/"
    logFiles = {}
    if not folder_path.startswith(logdir):
        folder_path = logdir + folder_path
    if not folder_path[-1] == "/":
        folder_path = folder_path + "/"
    
    for logfile in glob.glob("{}*.{}".format(folder_path,filetype)):
        filename = logfile.split("/")[-1]
        

        file = pd.read_csv(logfile)
        
        logFiles[filename[:-4]] = file
        
    return logFiles

data_50_best_fuel_max_end_set = load_logs("2021-02-11/JMetal-50-Nodes-Best-Selector/Total_Fuel_Max_Endpoints_set")
data_50_best_fuel_max = load_logs("2021-02-11/JMetal-50-Nodes-Best-Selector/Total_Fuel_Max_Endpoints_Not_set")
data_50_best_fuel_end_set = load_logs("2021-02-11/JMetal-50-Nodes-Best-Selector/Total_Fuel_Endpoints_set")
data_50_best_fuel = load_logs("2021-02-11/JMetal-50-Nodes-Best-Selector/Total_Fuel_Endpoints_Not_set")

data_50_random_fuel_max_end_set = load_logs("2021-02-11/JMetal-50-Nodes-Random-Selector/Total_Fuel_Max_Endpoints_set")
data_50_random_fuel_max = load_logs("2021-02-11/JMetal-50-Nodes-Random-Selector/Total_Fuel_Max_Endpoints_Not_set")
data_50_random_fuel_end_set = load_logs("2021-02-11/JMetal-50-Nodes-Random-Selector/Total_Fuel_Endpoints_set")
data_50_random_fuel = load_logs("2021-02-11/JMetal-50-Nodes-Random-Selector/Total_Fuel_Endpoints_Not_set")

data_10_fuel_max_end_set = load_logs("2021-02-11/JMetal-10-Nodes/Total_Fuel_Max_Endpoints_set")
data_10_fuel_max = load_logs("2021-02-11/JMetal-10-Nodes/Total_Fuel_Max_Endpoints_Not_set")
data_10_fuel_end_set = load_logs("2021-02-11/JMetal-10-Nodes/Total_Fuel_Endpoints_set")
data_10_fuel = load_logs("2021-02-11/JMetal-10-Nodes/Total_Fuel_Endpoints_Not_set")

google_10_nodes = load_logs("2021-02-11/OR-Tools-10-Nodes")
google_50_nodes = load_logs("2021-02-11/OR-Tools-50-Nodes")


In [120]:
dataFrames = [data_50_best_fuel,data_50_best_fuel_max,data_50_best_fuel_end_set,data_50_best_fuel_max_end_set,
 data_50_random_fuel,data_50_random_fuel_max,data_50_random_fuel_end_set,data_50_random_fuel_max_end_set,
 data_10_fuel_max_end_set,data_10_fuel_max,data_10_fuel_end_set,data_10_fuel,google_10_nodes,google_50_nodes]
for dataFrame in dataFrames:
    set_table_indexes(dataFrame)

In [121]:
import mplcursors
%matplotlib widget
#pylab
def plot_same_algorithm(algorithm_logs: dict,field:str,take_mean:False,plot_title:str,ylabel:str,nameDict:dict,xlabel="Iterations",ax=None):
    names = algorithm_logs.keys()
    ax = ax or plt.gca()
    bestVal = 10e10
    bestAlg = ""
    for name in names:
        plotData = algorithm_logs[name]
        
        if take_mean:
            plotData = plotData.to_numpy()
            plotData = np.array([literal_eval(plotData[i]) for i in range(len(plotData))])
            plotData = list(np.mean(plotData,axis = 0))
            ax.plot(plotData,label="{}: {}".format(name,round(plotData[-1],2)),linewidth=2)
            if plotData[-1] < bestVal:
                bestAlg = name
                bestVal = plotData[-1]
            
        else:
            ax.plot(plotData,label=name)
        ax.legend(prop={'size': 12})
    if take_mean:
        plot_title = plot_title + "\n{}: {}".format(bestAlg,round(bestVal,2))
        
    ax.set_xlabel(xlabel,fontsize=16)
    ax.set_ylabel(ylabel,fontsize=16)
    ax.tick_params(labelsize=12)
    ax.set_title(plot_title,fontsize=16)
    mplcursors.cursor(ax)
    
def plot_algorithms(algorithm_logs: dict,field:str,take_mean:False,plot_title:str,ylabel:str,nameDict:dict,xlabel="Iterations",ax=None):
    names = list(filter(lambda x: "Observer" in x,algorithm_logs.keys()))
    ax = ax or plt.gca()
    bestVal = 10e10
    bestAlg = ""
    for name in names:
        plotData = algorithm_logs[name][field]
        
        if take_mean:
            plotData = plotData.to_numpy()
            plotData = np.array([literal_eval(plotData[i]) for i in range(len(plotData))])
            plotData = list(np.mean(plotData,axis = 0))
            ax.plot(plotData,label="{}: {}".format(nameDict[name],round(plotData[-1],2)),linewidth=2)
            if plotData[-1] < bestVal:
                bestAlg = nameDict[name]
                bestVal = plotData[-1]
            
        else:
            ax.plot(plotData,label=nameDict[name])
        ax.legend(prop={'size': 12})
    if take_mean:
        plot_title = plot_title + "\n{}: {}".format(bestAlg,round(bestVal,2))
    ax.set_xlabel(xlabel,fontsize=16)
    ax.set_ylabel(ylabel,fontsize=16)
    ax.tick_params(labelsize=12)
    ax.set_title(plot_title,fontsize=16)
    mplcursors.cursor(ax)
def extract_algorithm_data(dataDicts:list,fieldName:str,algorithmName:str,newColumnNames:list):
    dataFields = {}
    for columnName,dataDict in zip(newColumnNames,dataDicts):
        dataFields[columnName]=(dataDict[algorithmName][fieldName])
    return pd.DataFrame(dataFields)

# Plot JMetal Comparisons between Algorithms for 10 Nodes

In [124]:

    
fig,axs = plt.subplots(2,2,figsize=(15,15))
plt.style.use('seaborn')

nameDict = {"Observer_SA":"SA","Observer_LS":"LS","Observer_NSGAIII":"NSGAIII","Observer_NSGAII":"NSGAII","Observer_GA":"GA"}

plot_algorithms(data_10_fuel_max_end_set,'fitness',True,"Fitness","Fitness",nameDict,ax=axs[0][0])
plot_algorithms(data_10_fuel_max_end_set,'fuel_consumption',True,"Fuel Consumption","Fuel Consumption",nameDict,ax=axs[0][1])
plot_algorithms(data_10_fuel_max_end_set,'computation_time',False,"Computation Time","Computation Time",nameDict,ax=axs[1][0])
plot_algorithms(data_10_fuel_max_end_set,'driveTimeDifferences',True,"Drive Times","Vehicle Road Time",nameDict,ax=axs[1][1])
axs[0][0].set_ylim(140,250)
axs[0][1].set_ylim(60,80)
plt.show()
#not looking for difference in drivetime, better to plot the highest drive time


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plot JMetal comparison between algorithms 50 Nodes with random selection for generation algorithms

In [26]:

    
fig,axs = plt.subplots(2,2,figsize=(15,15))
plt.style.use('seaborn')

nameDict = {"Observer_SA":"SA","Observer_LS":"LS","Observer_NSGAIII":"NSGAIII","Observer_NSGAII":"NSGAII","Observer_GA":"GA"}

plot_algorithms(data_50_random_fuel_max_end_set,'fitness',True,"Fitness","Fitness",nameDict,ax=axs[0][0])
plot_algorithms(data_50_random_fuel_max_end_set,'fuel_consumption',True,"Fuel Consumption","Fuel Consumption",nameDict,ax=axs[0][1])
plot_algorithms(data_50_random_fuel_max_end_set,'computation_time',False,"Computation Time","Computation Time",nameDict,ax=axs[1][0])
plot_algorithms(data_50_random_fuel_max_end_set,'driveTimeDifferences',True,"Drive Times","Vehicle Road Time",nameDict,ax=axs[1][1])
axs[0][0].set_ylim(180,600)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Plot JMetal comparison between algorithms 50 Nodes with best selection for generation algorithms

In [30]:
fig,axs = plt.subplots(2,2,figsize=(15,15))
plt.style.use('seaborn')

nameDict = {"Observer_SA":"SA","Observer_LS":"LS","Observer_NSGAIII":"NSGAIII","Observer_NSGAII":"NSGAII","Observer_GA":"GA"}

plot_algorithms(data_50_best_fuel_max_end_set,'fitness',True,"Fitness","Fitness",nameDict,ax=axs[0][0])
plot_algorithms(data_50_best_fuel_max_end_set,'fuel_consumption',True,"Fuel Consumption","Fuel Consumption",nameDict,ax=axs[0][1])
plot_algorithms(data_50_best_fuel_max_end_set,'computation_time',False,"Computation Time","Computation Time",nameDict,ax=axs[1][0])
plot_algorithms(data_50_best_fuel_max_end_set,'driveTimeDifferences',True,"Drive Times","Vehicle Road Time",nameDict,ax=axs[1][1])
axs[0][0].set_ylim(180,400)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Compare  algorithm versions for 50 nodes

In [31]:
fig,axs = plt.subplots(2,2,figsize=(15,15))
plt.style.use('seaborn')

SA_data = [data_50_best_fuel,data_50_best_fuel_max,data_50_best_fuel_end_set,data_50_best_fuel_max_end_set]
SA_50_comparison_fitness = extract_algorithm_data(SA_data,
                                          "fitness","Observer_SA",["Total Fuel","Total and Max Fuel","Total Fuel End Set","Total and Max Fuel End Set"])

SA_50_comparison_fuel = extract_algorithm_data(SA_data,
                                          "fuel_consumption","Observer_SA",["Total Fuel","Total and Max Fuel","Total Fuel End Set","Total and Max Fuel End Set"])

SA_50_comparison_computation_time = extract_algorithm_data(SA_data,
                                          "computation_time","Observer_SA",["Total Fuel","Total and Max Fuel","Total Fuel End Set","Total and Max Fuel End Set"])

SA_50_comparison_drive_time = extract_algorithm_data(SA_data,
                                          "meanDriveTimes","Observer_SA",["Total Fuel","Total and Max Fuel","Total Fuel End Set","Total and Max Fuel End Set"])

plot_same_algorithm(SA_50_comparison_fitness,'fitness',True,"Fitness","Fitness",nameDict,ax=axs[0][0])
plot_same_algorithm(SA_50_comparison_fuel,'fuel_consumption',True,"Fuel Consumption","Fuel Consumption",nameDict,ax=axs[0][1])
plot_same_algorithm(SA_50_comparison_computation_time,'computation_time',False,"Computation Time","Computation Time",nameDict,ax=axs[1][0])
plot_same_algorithm(SA_50_comparison_drive_time,'driveTimeDifferences',True,"Drive Times","Vehicle Road Time",nameDict,ax=axs[1][1])
plt.suptitle("Simulated Annealing 50 Nodes",fontsize=20)
axs[0][0].set_ylim(120,400)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
fig,axs = plt.subplots(2,2,figsize=(15,15))
plt.style.use('seaborn')


Data_50_nodes = [data_50_best_fuel,data_50_best_fuel_max,data_50_best_fuel_end_set,data_50_best_fuel_max_end_set]

LS_50_comparison_fitness = extract_algorithm_data(Data_50_nodes,
                                          "fitness","Observer_LS",["Total Fuel","Total and Max Fuel","Total Fuel End Set","Total and Max Fuel End Set"])

LS_50_comparison_fuel = extract_algorithm_data(Data_50_nodes,
                                          "fuel_consumption","Observer_LS",["Total Fuel","Total and Max Fuel","Total Fuel End Set","Total and Max Fuel End Set"])

LS_50_comparison_computation_time = extract_algorithm_data(Data_50_nodes,
                                          "computation_time","Observer_LS",["Total Fuel","Total and Max Fuel","Total Fuel End Set","Total and Max Fuel End Set"])

LS_50_comparison_drive_time = extract_algorithm_data(Data_50_nodes,
                                          "meanDriveTimes","Observer_LS",["Total Fuel","Total and Max Fuel","Total Fuel End Set","Total and Max Fuel End Set"])



plot_same_algorithm(LS_50_comparison_fitness,'fitness',True,"Fitness","Fitness",nameDict,ax=axs[0][0])
plot_same_algorithm(LS_50_comparison_fuel,'fuel_consumption',True,"Fuel Consumption","Fuel Consumption",nameDict,ax=axs[0][1])
plot_same_algorithm(LS_50_comparison_computation_time,'computation_time',False,"Computation Time","Computation Time",nameDict,ax=axs[1][0])
plot_same_algorithm(LS_50_comparison_drive_time,'driveTimeDifferences',True,"Drive Times","Vehicle Road Time",nameDict,ax=axs[1][1])
axs[0][0].set_ylim(120,400)
plt.suptitle("Local Search 50 Nodes",fontsize=20)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [33]:
fig,axs = plt.subplots(2,2,figsize=(15,15))
plt.style.use('seaborn')


Data_50_nodes = [data_50_best_fuel,data_50_best_fuel_max,data_50_best_fuel_end_set,data_50_best_fuel_max_end_set,
                     data_50_random_fuel,data_50_random_fuel_max,data_50_random_fuel_end_set,data_50_random_fuel_max_end_set]

columnNames = ["Best:Total Fuel","Best:Total and Max Fuel","Best:Total Fuel End Set","Best:Total and Max Fuel End Set",
               "Random:Total Fuel","Random:Total and Max Fuel","Random:Total Fuel End Set","Random:Total and Max Fuel End Set"]

NSGAIII_50_comparison_fitness = extract_algorithm_data(Data_50_nodes,
                                          "fitness","Observer_NSGAIII",columnNames)

NSGAIII_50_comparison_fuel = extract_algorithm_data(Data_50_nodes,
                                          "fuel_consumption","Observer_NSGAIII",columnNames)

NSGAIII_50_comparison_computation_time = extract_algorithm_data(Data_50_nodes,
                                          "computation_time","Observer_NSGAIII",columnNames)

NSGAIII_50_comparison_drive_time = extract_algorithm_data(Data_50_nodes,
                                          "meanDriveTimes","Observer_NSGAIII",columnNames)



plot_same_algorithm(NSGAIII_50_comparison_fitness,'fitness',True,"Fitness","Fitness",nameDict,ax=axs[0][0])
plot_same_algorithm(NSGAIII_50_comparison_fuel,'fuel_consumption',True,"Fuel Consumption","Fuel Consumption",nameDict,ax=axs[0][1])
plot_same_algorithm(NSGAIII_50_comparison_computation_time,'computation_time',False,"Computation Time","Computation Time",nameDict,ax=axs[1][0])
plot_same_algorithm(NSGAIII_50_comparison_drive_time,'driveTimeDifferences',True,"Drive Times","Vehicle Road Time",nameDict,ax=axs[1][1])
axs[0][0].set_ylim(120,400)
plt.suptitle("NSGAIII 50 Nodes",fontsize=20)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
fig,axs = plt.subplots(2,2,figsize=(15,15))
plt.style.use('seaborn')


Data_50_nodes = [data_50_best_fuel,data_50_best_fuel_max,data_50_best_fuel_end_set,data_50_best_fuel_max_end_set,
                     data_50_random_fuel,data_50_random_fuel_max,data_50_random_fuel_end_set,data_50_random_fuel_max_end_set]

columnNames = ["Best:Total Fuel","Best:Total and Max Fuel","Best:Total Fuel End Set","Best:Total and Max Fuel End Set",
               "Random:Total Fuel","Random:Total and Max Fuel","Random:Total Fuel End Set","Random:Total and Max Fuel End Set"]

NSGAII_50_comparison_fitness = extract_algorithm_data(Data_50_nodes,
                                          "fitness","Observer_NSGAII",columnNames)

NSGAII_50_comparison_fuel = extract_algorithm_data(Data_50_nodes,
                                          "fuel_consumption","Observer_NSGAII",columnNames)

NSGAII_50_comparison_computation_time = extract_algorithm_data(Data_50_nodes,
                                          "computation_time","Observer_NSGAII",columnNames)

NSGAII_50_comparison_drive_time = extract_algorithm_data(Data_50_nodes,
                                          "meanDriveTimes","Observer_NSGAII",columnNames)



plot_same_algorithm(NSGAII_50_comparison_fitness,'fitness',True,"Fitness","Fitness",nameDict,ax=axs[0][0])
plot_same_algorithm(NSGAII_50_comparison_fuel,'fuel_consumption',True,"Fuel Consumption","Fuel Consumption",nameDict,ax=axs[0][1])
plot_same_algorithm(NSGAII_50_comparison_computation_time,'computation_time',False,"Computation Time","Computation Time",nameDict,ax=axs[1][0])
plot_same_algorithm(NSGAII_50_comparison_drive_time,'driveTimeDifferences',True,"Drive Times","Vehicle Road Time",nameDict,ax=axs[1][1])
axs[0][0].set_ylim(120,400)
plt.suptitle("NSGAII 50 Nodes",fontsize=20)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
fig,axs = plt.subplots(2,2,figsize=(15,15))
plt.style.use('seaborn')


Data_50_nodes = [data_50_best_fuel,data_50_best_fuel_max,data_50_best_fuel_end_set,data_50_best_fuel_max_end_set,
                     data_50_random_fuel,data_50_random_fuel_max,data_50_random_fuel_end_set,data_50_random_fuel_max_end_set]

columnNames = ["Best:Total Fuel","Best:Total and Max Fuel","Best:Total Fuel End Set","Best:Total and Max Fuel End Set",
               "Random:Total Fuel","Random:Total and Max Fuel","Random:Total Fuel End Set","Random:Total and Max Fuel End Set"]

GA_50_comparison_fitness = extract_algorithm_data(Data_50_nodes,
                                          "fitness","Observer_GA",columnNames)

GA_50_comparison_fuel = extract_algorithm_data(Data_50_nodes,
                                          "fuel_consumption","Observer_GA",columnNames)

GA_50_comparison_computation_time = extract_algorithm_data(Data_50_nodes,
                                          "computation_time","Observer_GA",columnNames)

GA_50_comparison_drive_time = extract_algorithm_data(Data_50_nodes,
                                          "meanDriveTimes","Observer_GA",columnNames)



plot_same_algorithm(GA_50_comparison_fitness,'fitness',True,"Fitness","Fitness",nameDict,ax=axs[0][0])
plot_same_algorithm(GA_50_comparison_fuel,'fuel_consumption',True,"Fuel Consumption","Fuel Consumption",nameDict,ax=axs[0][1])
plot_same_algorithm(GA_50_comparison_computation_time,'computation_time',False,"Computation Time","Computation Time",nameDict,ax=axs[1][0])
plot_same_algorithm(GA_50_comparison_drive_time,'driveTimeDifferences',True,"Drive Times","Vehicle Road Time",nameDict,ax=axs[1][1])
axs[0][0].set_ylim(120,400)
plt.suptitle("GA 50 Nodes",fontsize=20)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Show final tables

# Comparison between Google and JMetal, 10 Nodes

In [113]:
from ipywidgets import widgets, Layout
from IPython import display
import pandas as pd
import numpy as np

# create output widgets
widget1 = widgets.Output()
widget2 = widgets.Output()
widget3 = widgets.Output()

# render in output widgets
with widget1:
    display.display(google_10_nodes['Final_SA'])
with widget2:
    display.display(data_10_fuel['Final_SA'])
with widget3:
    display.display(data_10_fuel_end_set['Final_SA'])


# add some CSS styles to distribute free space
box_layout = Layout(display='flex',
                    flex_flow='row',
                    justify_content='space-around',
                    width='auto'
                   )
    
# create Horisontal Box container
hbox = widgets.HBox([widget1, widget2,widget3], layout=box_layout)
print("================================================Google SA=========================================================================JMETAL Total fuel SA==================================================JMETAL Total fuel and end set SA=======")
# render hbox
hbox

================================================Google SA=========================================================================JMETAL Total fuel SA==================================================JMETAL Total fuel and end set SA=======


# Comparison between Google and JMetal, 50 Nodes

In [114]:
from ipywidgets import widgets, Layout
from IPython import display
import pandas as pd
import numpy as np

# create output widgets
widget1 = widgets.Output()
widget2 = widgets.Output()
widget3 = widgets.Output()

# render in output widgets
with widget1:
    display.display(google_50_nodes['Final_SA'])
with widget2:
    display.display(data_50_best_fuel['Final_SA'])
with widget3:
    display.display(data_50_best_fuel_end_set['Final_SA'])


# add some CSS styles to distribute free space
box_layout = Layout(display='flex',
                    flex_flow='row',
                    justify_content='space-around',
                    width='auto'
                   )
    
# create Horisontal Box container
hbox = widgets.HBox([widget1, widget2,widget3], layout=box_layout)
print("================================================Google SA=========================================================================JMETAL Total fuel SA==================================================JMETAL Total fuel and end set SA=======")
# render hbox
hbox

================================================Google SA=========================================================================JMETAL Total fuel SA==================================================JMETAL Total fuel and end set SA=======


# Comparison between Google GLS and JMetal LS, 50 Nodes

In [118]:
from ipywidgets import widgets, Layout
from IPython import display
import pandas as pd
import numpy as np

# create output widgets
widget1 = widgets.Output()
widget2 = widgets.Output()
widget3 = widgets.Output()

# render in output widgets
with widget1:
    display.display(google_50_nodes['Final_GLS'])
with widget2:
    display.display(data_50_best_fuel['Final_LS'])
with widget3:
    display.display(data_50_best_fuel_end_set['Final_LS'])


# add some CSS styles to distribute free space
box_layout = Layout(display='flex',
                    flex_flow='row',
                    justify_content='space-around',
                    width='auto'
                   )
    
# create Horisontal Box container
hbox = widgets.HBox([widget1, widget2,widget3], layout=box_layout)
print("================================================Google GLS=========================================================================JMETAL Total fuel LS===================================================JMETAL Total fuel and end set LS======")
# render hbox
hbox

================================================Google GLS=========================================================================JMETAL Total fuel LS===================================================JMETAL Total fuel and end set LS======


# Comparison between different Google algorithms, 50 Nodes

In [116]:
from ipywidgets import widgets, Layout
from IPython import display
import pandas as pd
import numpy as np

# create output widgets
widget1 = widgets.Output()
widget2 = widgets.Output()
widget3 = widgets.Output()

# render in output widgets
with widget1:
    display.display(google_50_nodes['Final_SA'])
with widget2:
    display.display(google_50_nodes['Final_TABU'])
with widget3:
    display.display(google_50_nodes['Final_GLS'])


# add some CSS styles to distribute free space
box_layout = Layout(display='flex',
                    flex_flow='row',
                    justify_content='space-around',
                    width='auto'
                   )
    
# create Horisontal Box container
hbox = widgets.HBox([widget1, widget2,widget3], layout=box_layout)
print("================================================Google SA=========================================================================Google TABU===================================================================Google GLS=======")
# render hbox
hbox

================================================Google SA=========================================================================Google TABU===================================================================Google GLS=======
